# 🧪 Test LlamaStack dans OpenShift AI

Ce notebook teste la fonctionnalité LlamaStack avec le modèle Llama-3.2-1B-Instruct déployé sur OpenShift AI.

## 📋 Prérequis
- Namespace: `llama-instruct-32-1b-demo`
- LlamaStackDistribution: `lsd-llama-32-1b-instruct`
- InferenceService: `llama-32-1b-instruct`

## 🔗 Services utilisés
- `lsd-llama-32-1b-instruct-service:8321` (LlamaStack)
- `llama-32-1b-instruct-predictor:80` (vLLM)

In [ ]:
# Installation des dépendances
!pip install llama-stack-client fire requests

In [ ]:
# Import des librairies
import requests
import json
from llama_stack_client import Client, Agent, AgentEventLogger
import uuid
import os

print("✅ Librairies importées")

## 🔍 Test 1: Vérification des services

In [ ]:
# Test de connexion à LlamaStack
llamastack_url = "http://lsd-llama-32-1b-instruct-service:8321"

print(f"🔗 Test de connexion à LlamaStack: {llamastack_url}")

try:
    response = requests.get(f"{llamastack_url}/v1/models", timeout=10)
    print(f"✅ Status: {response.status_code}")
    
    if response.status_code == 200:
        models = response.json()
        print("📋 Modèles disponibles:")
        for model in models.get('data', []):
            print(f"  - {model.get('identifier', 'N/A')} ({model.get('model_type', 'N/A')})")
    else:
        print(f"❌ Erreur: {response.text}")
        
except Exception as e:
    print(f"❌ Erreur de connexion: {e}")

In [ ]:
# Test de connexion à vLLM
vllm_url = "http://llama-32-1b-instruct-predictor:80"

print(f"🔗 Test de connexion à vLLM: {vllm_url}")

try:
    response = requests.get(f"{vllm_url}/v1/models", timeout=10)
    print(f"✅ Status: {response.status_code}")
    
    if response.status_code == 200:
        models = response.json()
        print("📋 Modèles vLLM disponibles:")
        for model in models.get('data', []):
            print(f"  - {model.get('id', 'N/A')}")
    else:
        print(f"❌ Erreur: {response.text}")
        
except Exception as e:
    print(f"❌ Erreur de connexion: {e}")

## 🤖 Test 2: Client LlamaStack

In [ ]:
# Connexion au client LlamaStack
try:
    client = Client(base_url=llamastack_url)
    print("✅ Client LlamaStack connecté")
    
    # Lister les modèles
    models = client.models.list()
    print(f"📋 {len(models)} modèles trouvés:")
    for model in models:
        print(f"  - {model.identifier} ({model.model_type})")
        
except Exception as e:
    print(f"❌ Erreur client LlamaStack: {e}")

In [ ]:
# Vérifier les bases vectorielles
try:
    vector_dbs = client.vector_dbs.list()
    print(f"🗄️ {len(vector_dbs)} bases vectorielles trouvées:")
    for db in vector_dbs:
        print(f"  - {db.identifier}")
        
    if not vector_dbs:
        print("ℹ️ Aucune base vectorielle trouvée (normal si pas encore d'ingestion)")
        
except Exception as e:
    print(f"❌ Erreur bases vectorielles: {e}")

## 💬 Test 3: Agent LlamaStack

In [ ]:
# Créer un agent LlamaStack
try:
    # Trouver le modèle LLM
    llm_model = None
    for model in models:
        if model.model_type == "llm":
            llm_model = model
            break
    
    if llm_model:
        print(f"🤖 Modèle LLM trouvé: {llm_model.identifier}")
        
        # Créer un agent simple (sans RAG pour l'instant)
        agent = Agent(
            client,
            model=llm_model.identifier,
            instructions="Tu es un assistant IA utile et amical."
        )
        print("✅ Agent LlamaStack créé")
        
        # Test simple
        session_id = agent.create_session(session_name=f"test_session_{uuid.uuid4().hex[:8]}")
        print(f"📝 Session créée: {session_id}")
        
        # Question simple
        response = agent.create_turn(
            messages=[{"role": "user", "content": "Bonjour, comment allez-vous ?"}],
            session_id=session_id,
            stream=True
        )
        
        print("💬 Réponse:")
        for log in AgentEventLogger().log(response):
            if hasattr(log, 'content') and log.content:
                print(log.content, end='', flush=True)
        print()
        
    else:
        print("❌ Aucun modèle LLM trouvé")
        
except Exception as e:
    print(f"❌ Erreur agent: {e}")

## 🔧 Test 4: Test direct vLLM

In [ ]:
# Test direct de l'API vLLM
print("🔧 Test direct de l'API vLLM")

try:
    # Test de chat completion
    chat_payload = {
        "model": "llama-32-1b-instruct",
        "messages": [
            {"role": "user", "content": "Explique-moi ce qu'est l'intelligence artificielle en 2 phrases."}
        ],
        "max_tokens": 150,
        "temperature": 0.7
    }
    
    response = requests.post(
        f"{vllm_url}/v1/chat/completions",
        json=chat_payload,
        timeout=30
    )
    
    if response.status_code == 200:
        result = response.json()
        print("✅ Réponse vLLM:")
        print(result['choices'][0]['message']['content'])
    else:
        print(f"❌ Erreur vLLM: {response.status_code}")
        print(response.text)
        
except Exception as e:
    print(f"❌ Erreur test vLLM: {e}")

## 📊 Résumé des tests

In [ ]:
print("🎉 Tests terminés !")
print("\n📋 Prochaines étapes:")
print("1. Si les tests passent, déployer la pipeline d'ingestion RAG")
print("2. Tester la fonctionnalité RAG avec des documents")
print("3. Utiliser le use case assurance")
print("\n🔗 Services testés:")
print(f"  - LlamaStack: {llamastack_url}")
print(f"  - vLLM: {vllm_url}")
print("\n📚 Documentation:")
print("  - README RAG: llamastack/rag/README.md")
print("  - Scripts de déploiement: deploy-rag.sh, deploy-assurance.sh")